In [1]:
import math

import pandas as pd
import numpy as np
from keras import Input,backend
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score, accuracy_score, matthews_corrcoef

Using TensorFlow backend.


In [39]:
dataset = pd.read_csv("../datasets/train_test_data.csv",
                           header=0, parse_dates=[0], index_col=0)

In [40]:
#hyperparamters
batch_size=64

#params for generator
label_index = len(dataset.columns) -1
delay=0
step=1 # 1 timestep = 1 day
lookback=10

#ratio for train/val/test split
train_ratio=0.7
val_ratio=0.15

train_max_idx = math.ceil(len(dataset)*train_ratio)
val_max_idx = math.ceil(len(dataset)*(train_ratio+val_ratio))

# 1 step = 1 batche of samples 
train_steps = (train_max_idx+1) // batch_size
val_steps =  (val_max_idx - train_max_idx - lookback) // batch_size
test_steps = (len(dataset) - val_max_idx - lookback) // batch_size

In [6]:
def generator(data, label_index, lookback, delay, min_index, max_index,
              shuffle=False, batch_size=64, step=1):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback
    while 1:
        if shuffle:
            rows = np.random.randint(
                min_index + lookback, max_index, size=batch_size)
        else:
            if i + batch_size >= max_index: 
                i = min_index + lookback #reset 'i'
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)

        samples = np.zeros((len(rows),
                           lookback // step,
                           (data.shape[-1] - 1)))
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            samples[j] = np.delete(data[indices], label_index, axis=1) # an np array without the label col
            targets[j] = data[rows[j] + delay][label_index]
        yield samples, targets

In [7]:
train_rand_gen = generator(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=train_max_idx,
                      shuffle=True,
                      step=step, 
                      batch_size=batch_size)

train_gen = generator(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=train_max_idx,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)


val_gen = generator(dataset.to_numpy(),
                    label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=train_max_idx+1,
                      max_index=val_max_idx,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)

test_gen = generator(dataset.to_numpy(),
                      label_index=label_index,
                      lookback=lookback,
                      delay=delay,
                      min_index=val_max_idx+1,
                      max_index=None,
                      shuffle=False,
                      step=step, 
                      batch_size=batch_size)

In [8]:
# count = 0
# max_count=1
# for samples, targets in t_gen:
#     if(count >= max_count):
#         break
#     print(samples.shape)
#     print(targets)
#     print(samples[0])
#     print(samples[1])
#     count+=1

In [9]:
def print_metric(y_true, y_pred):
    print('F1 score: %f' % f1_score(y_true, y_pred))
    print('precision score: %f' % precision_score(y_true, y_pred))
    print('recall score: %f' % recall_score(y_true, y_pred))
    print('accuracy score: %f' % accuracy_score(y_true, y_pred))
    print('matthews_corrcoef: %f' % matthews_corrcoef(y_true, y_pred))
    print('\nConfusion matrix:')
    print(confusion_matrix(y_true, y_pred, labels=[0,1]))

In [10]:
def probs_to_binary_classes(preds, threshold=0.5):
    """
    preds: np array
    threshold: scalar
    """
    return np.where(preds > threshold, 1, 0)

In [11]:
def print_unique_counts(x):
    unique, counts = np.unique(x, return_counts=True)
    print(np.asarray((unique, counts)).T)


In [12]:
#build s FC model from the book

input_shape = (lookback//step, dataset.shape[-1] - 1)

model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

# #fit the model
model.fit_generator(train_rand_gen,
                    steps_per_epoch=train_steps,
                    epochs=100, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 250)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8032      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 8,065
Trainable params: 8,065
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/400
61/61 [==============================] - 0s 7ms/step - loss: 0.5419 - acc: 0.7451
Epoch 2/400
61/61 [==============================] - 0s 6ms/step - loss: 0.5354 - acc: 0.7398
Epoch 3/400
61/61 [==============================] - 0s 6ms/step - loss: 0.5036 - acc: 0.7556
Epoch 4/400
61/61 [==============================] - 0s 6ms/step - loss: 0.5111 - acc: 0.7536
Epoch 5/400
61/61 [====

61/61 [==============================] - 0s 4ms/step - loss: 0.3974 - acc: 0.8261
Epoch 80/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3805 - acc: 0.8379
Epoch 81/400
61/61 [==============================] - 0s 4ms/step - loss: 0.4068 - acc: 0.8240
Epoch 82/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3874 - acc: 0.8299
Epoch 83/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3910 - acc: 0.8299A: 0s - loss: 0.3975 - acc: 0.8
Epoch 84/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3895 - acc: 0.8289A: 0s - loss: 0.3911 - acc: 0.
Epoch 85/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3848 - acc: 0.8299
Epoch 86/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3795 - acc: 0.8412
Epoch 87/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3962 - acc: 0.8289
Epoch 88/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3930 - acc: 0.8315
Ep

61/61 [==============================] - 0s 4ms/step - loss: 0.3376 - acc: 0.8517
Epoch 163/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3278 - acc: 0.8609
Epoch 164/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3251 - acc: 0.8665
Epoch 165/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3471 - acc: 0.8522
Epoch 166/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3333 - acc: 0.8589
Epoch 167/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3438 - acc: 0.8453
Epoch 168/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3360 - acc: 0.8591A: 0s - loss: 0.3577 - acc: 0.
Epoch 169/400
61/61 [==============================] - 0s 5ms/step - loss: 0.3380 - acc: 0.8517
Epoch 170/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3372 - acc: 0.8563
Epoch 171/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3535 - acc: 0.8476
Epoch 172/400
61/61 [===

61/61 [==============================] - 0s 4ms/step - loss: 0.2980 - acc: 0.8763
Epoch 248/400
61/61 [==============================] - 0s 4ms/step - loss: 0.3010 - acc: 0.8681
Epoch 249/400
61/61 [==============================] - 0s 4ms/step - loss: 0.2961 - acc: 0.8778
Epoch 250/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2853 - acc: 0.8832
Epoch 251/400
61/61 [==============================] - 0s 5ms/step - loss: 0.3075 - acc: 0.8660
Epoch 252/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2967 - acc: 0.8773
Epoch 253/400
61/61 [==============================] - 0s 5ms/step - loss: 0.3069 - acc: 0.8683
Epoch 254/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2871 - acc: 0.8753
Epoch 255/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2840 - acc: 0.8801
Epoch 256/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2770 - acc: 0.8835
Epoch 257/400
61/61 [==============================] -

61/61 [==============================] - 0s 5ms/step - loss: 0.2592 - acc: 0.8937
Epoch 333/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2645 - acc: 0.8845
Epoch 334/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2699 - acc: 0.8886
Epoch 335/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2583 - acc: 0.8901
Epoch 336/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2694 - acc: 0.8870
Epoch 337/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2449 - acc: 0.8991
Epoch 338/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2593 - acc: 0.8888
Epoch 339/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2638 - acc: 0.8870
Epoch 340/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2519 - acc: 0.9034
Epoch 341/400
61/61 [==============================] - 0s 5ms/step - loss: 0.2550 - acc: 0.8993
Epoch 342/400
61/61 [==============================] -

In [77]:
def get_data_from_generator(generator, steps):
    count=0
    X, Y = [], []
    for samples, targets in generator:
        if count >= steps:
            break;
        else:
            count+=1

        X.append(samples)
        Y.append(targets)

    return np.concatenate(X, axis=0), np.concatenate(Y, axis=0)


In [84]:
neg = 2762+158
pos = 177+807
print("We have {0} neg cases and {1} pos cases from train data".format(neg,pos))
print("the common sense baseline (accuracy score) is {0}".format(neg/(pos+neg)))

We have 2920 neg cases and 984 pos cases from train data
the common sense baseline (accuracy score) is 0.7479508196721312


In [79]:
#Make predictions for train set
    
X_train, Y_train = get_data_from_generator(train_gen, train_steps)    
y_pred = model.predict(X)
print_metric(Y, probs_to_binary_classes(y_pred))

F1 score: 0.828117
precision score: 0.836269
recall score: 0.820122
accuracy score: 0.914191
matthews_corrcoef: 0.771012

Confusion matrix:
[[2762  158]
 [ 177  807]]


In [80]:
#Make predictions from dev set
X, Y = get_data_from_generator(val_gen, val_steps)    
y_pred = model.predict(X)
print_metric(Y, probs_to_binary_classes(y_pred))


F1 score: 0.081633
precision score: 0.045977
recall score: 0.363636
accuracy score: 0.891827
matthews_corrcoef: 0.098000

Confusion matrix:
[[738  83]
 [  7   4]]


In [81]:
#Make predictions for test set
X, Y = get_data_from_generator(test_gen, test_steps)    
y_pred = model.predict(X)
print_metric(Y, probs_to_binary_classes(y_pred))


F1 score: 0.000000
precision score: 0.000000
recall score: 0.000000
accuracy score: 0.854567
matthews_corrcoef: -0.067777

Confusion matrix:
[[711  91]
 [ 30   0]]
